In [ ]:
"""
BRUTE FORCE  -> грубая сила соглашение о поиске <-

param :[w]:t[] 
param return

"""

@timeit
@memoize
def syukaj(w ,t):
    i ,j = 0, 0
    M = len(w)
    N = len(t)
    while j < M and i < N:
        if t[i] != w[j]:
            i = i-(j-i)
            j=-1
        i=i+1
        j=j+1
    if j==M:
        return i-M
    else:
        return -1
    
"""
KMP
"""
@timeit
@memoize
shift=list()
def kmp(w, t):
    N=len(t)
    M=len(w)
    i, j = 0, 0
    while i<N and j<M:
        while j>=0 and t[i]!=w[j]:
            j=shift[j]
        i=i+1
        j=j+1
    if j==M:
        return i-M
    else:
        return -1
@timeit
@memoize
def init_shifts(w):
    M=len(w)
    global shift
    shift = [0]*len(w)
    shift[0]=-1
    i, j = 0, -1
    while i < M -1:
        shift[i] = j
        while j >= 0 and w[i]!=w[j]:
            j=shift[j]
        i = i + 1
        j = j + 1


In [ ]:
from flask import Flask, render_template, request

from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import re 
import os
import pickle
import requests 

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_caterogical
from keras.models import Sequential
from keras.layers import Flatten, Dense

import matplotlib.pyplot as plt

In [15]:
app = Flask(__name__)

class ReviewForm(Form):
    shop_review = TextAreaField('',
                                [validator.DataRequired(),
                                 validators.length(min=15)])

@app.route('/')
def index():
    return 0

@app.route('/results', methods=['POST'])
def results():
    form = ReviewForm(request.form)
    if request.method == 'POST' and form.validate():
        review = request.form['shop_review']
        y, proba = classify(review)
        return render_template('results.html',
                               content=review,
                               prediction=y,
                               probability=round(proba*100, 2))
    return render_template('reviewform.html', form=form)

NameError: name 'Form' is not defined

In [11]:
review_dir = ' '
train_dir = os.path.join(review_dir, 'train')

labels = []
texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == ".txt":
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label == 'neg':
                labels.append(0)
            else:
                labels.append(1)

FileNotFoundError: [Errno 2] No such file or directory: ' /train/neg'

In [ ]:
cur_dir = os.path.dirname(__file__)

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.finall('(?::|;|=)(?:-\)|\(|D|P)',
                          text.lowe())
    text = re.sub('[\W]+', ' ', text.lower()) \
    + ' '.join(emoticons).replace('-', '')
    
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)
                 

In [12]:
df = pd.read_csv('reviews_datase.tsv.zip', header=0, delimeter='\t', quoting=3)
df = df[['review','sentiment']]
#df.shape()

df.sentiment.value_counts()

# Preprocessing using regular expressions
df['review'] = df['review'].apply(lambda x: x.lower())
df['review'] = df['review'].apply((lambda x: re.sub('a-zA-z0-9\s]','',x)))

maxlen = 100
training_samples = 200
validation_samples = 10000
max_words = 10000

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)

sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)
indices = np.arrange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

ModuleNotFoundError: No module named 'keras'

In [13]:
embedding_index = {}
f = open(os.path.join(dir, ' '))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embedding_index[word] = coefs
f.close()

embedding_dim = 100

embedding_matrix = np,zeros((max_words, embedding))
for word, i in word_index_items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            

TypeError: expected str, bytes or os.PathLike object, not builtin_function_or_method

In [14]:
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy'
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val, y_val))

model.save_weights('pre_trained_glove_model.h5')

SyntaxError: invalid syntax (1739870763.py, line 16)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_acc']
loss = history.history['loss']

val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

"""
plots
"""